# Аспектный анализ тональности текстов

* [1] https://github.com/nlpub/pymystem3
* [2] SentiRuEval2015
* [3] https://rusvectores.org/ru/models/

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle,
# выполните следующие строчки, чтобы подгрузить библиотеку dlnlputils:

!git clone https://github.com/Samsung-IT-Academy/stepik-dl-nlp.git && pip install -r stepik-dl-nlp/requirements.txt
import sys; sys.path.append('./stepik-dl-nlp')

Cloning into 'stepik-dl-nlp'...
remote: Enumerating objects: 293, done.
remote: Total 293 (delta 0), reused 0 (delta 0), pack-reused 293
Receiving objects: 100% (293/293), 42.29 MiB | 18.67 MiB/s, done.
Resolving deltas: 100% (134/134), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 1.7 MB 47.7 MB/s 
     |████████████████████████████████| 23.4 MB 4.9 MB/s 
     |████████████████████████████████| 304 kB 68.0 MB/s 
     |████████████████████████████████| 8.2 MB 57.1 MB/s 
     |████████████████████████████████| 1.6 MB 58.2 MB/s 
  Created wheel for gensim: filename=gensim-3.8.1-cp38-cp38-linux_x86_64.whl size=25870879 sha256=2b98098d6c9b7610780d630c952db501004b3bea511a791672a17af982cbcddf
  Stored in directory: /root/.cache/pip/wheels/33/de/03/7346ae70da7f980f78569668caf78fb2d678b176e549557c7d
  Created wheel for ufal.udpipe: filename=

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

torch.manual_seed(1)

## Оригинальная разметка

In [ ]:
!wget 'https://raw.githubusercontent.com/suppukerr/project_auto/main/aspects_bio.tsv'

In [ ]:
from dlnlputils.sentiment_utils import parse_xml_sentiment, parse_xml_aspect, show_markup

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


тексты с разметкой аспектов и тональностей: 

In [ ]:
texts_w_sentiment_spans = parse_xml_sentiment(xml_sentiments)
texts_w_aspect_spans    = parse_xml_aspect(xml_sentiments)

FileNotFoundError: ignored

In [ ]:
amount = len(texts_w_sentiment_spans)

print('Загружено {} текстов с разметкой тональности\n'.format(amount))

NameError: ignored

##### выберем 2 текста, на которых будем рассматривать все примеры:

In [ ]:
random_picks = [random.randint(0,amount-1) for _ in range(0,2)]

####  тональность (sentiment)

In [ ]:
for rand_i in random_picks:
    text, spans = texts_w_sentiment_spans[rand_i]
    
    print('Текст №:',rand_i)
    show_markup(text,spans)

Текст №: 76


BoxMarkup('Сегодня я купил машину лада гранта и ни как не могу ее продать начнем с покупки машина мне обошлась в четыреста восемдесят тысяч рублей на реканий на машину нет стоимость масла 850 рублей замена бампера 5000 рублей если собираетесь делать ремонт машины капитально, то стоимость обойдется в районе 200000 рублей мощьность у нее 100 лошадей макс скорость 185 км в час. Общее впечатление : замечательный автомобиль',
          [Span(16, 34, 'neutral'),
           Span(80, 86, 'neutral'),
           Span(129, 135, 'neutral'),
           Span(150, 156, 'positive'),
           Span(161, 176, 'neutral'),
           Span(208, 214, 'neutral'),
           Span(239, 252, 'neutral'),
           Span(268, 277, 'neutral'),
           Span(311, 320, 'neutral'),
           Span(331, 338, 'neutral'),
           Span(339, 352, 'neutral'),
           Span(401, 411, 'positive')])

Текст №: 43


BoxMarkup('Начну с того, что на Тойоте Хайс работает мой муж и уже давненько. Поначалу машинка очень понравилась и внешне, и внутренне. Муж был просто в восторге, что его по работе пересадили с Баргузина на Хайс, и не замечал недостатков в этой машине. А так как он периодически отвозит меня на работу и забирает с работы на ней, то и я волей-неволей столкнулась с этой машиной. Конечно, после Баргузина Хайс казался просто нереальной машиной. И удобство кресел в салоне, и плавность хода, и внешний вид, всё приводило в восторг. Муж даже начал бредить идеей, что для нашей большой семьи (у нас трое деток) покупка такой машины в семью будет идеальной. Даже в мечтах обрисовывал, как он переделает салон под междугородние путешествия всей семьёй. Но прошло уже года три-четыре, как он работает на Тойоте Хайс. Сейчас могу сказать только одно: машина ужасная до безобразия. В этом мы убедились оба. Начну с того, что проживаем мы в северном районе нашей страны, и, соответственно, машина эксплуатируется зимой при довольно низких температурах. Так вот, в зимний период салон прогревается только в зоне передних сидений (водителя и пассажира), и то кое-как. В самом салоне машина не прогревается, несмотря на дополнительные печки и всевозможные регуляторы температуры нагрева. Так же зимой на поворотах (например, на перекрёстке) очень сильно заносит заднюю часть автомобиля. Ну это понятно - задний привод и всё остальное прочее. Но заносы эти, даже глядя со стороны, приводят в тихую жуть. Хайсов по городу бегает достаточное количество и регулярно наблюдается картина с этими заносами. Дверь на салазках снаружи открывается ужасно - женская половина пассажиров, в том числе и я, регулярно ломают ногти, так как ручка сделана не очень хорошо и с неё постоянно соскальзывают пальцы. В водительской части салона в принципе всё довольно неплохо, за исключением торпеды, которая после недолгой эксплуатации автомобиля начала греметь, что само по себе уже не приятно. Да и сам кузов Хайса начал скрипеть, чем сразу заслужил гордое звание "Сарай". Сервисное обслуживание в Тойота-центре нашего города тоже ниже плинтуса. Машина хоть и обслуживается за счёт организации, но после каждого ТО мой муж, мягко говоря, в шоке от того, как сервис-менеджеры каждый раз находят на ровном месте какие-то "болячки" у машины, дабы содрать побольше денег. А те поломки, которые действительно требуют серьёзного внимания, в заказ-наряде прописываются, но специалисты даже и не заглядывают туда. Таким образом, мой муж половину этой зимы проездил с "инвалидным" задним правым колесом: с места тронуться было невозможно - оно просто не крутилось, приходилось его срывать. Вобщем, впечатление от Хайса не айс (прям каламбур получился). Внешне, да, выглядит красиво и величественно. Но на самом деле такой же "сарай" как Баргузин, Газель и прочее. Одно, что кочки пятой точкой не чувствуешь :) За такую цену ожидаешь лучшего качества. ',
          [Span(21, 32, 'neutral'),
           Span(76, 83, 'positive'),
           Span(104, 110, 'positive'),
           Span(183, 192, 'neutral'),
           Span(196, 200, 'positive'),
           Span(234, 240, 'both'),
           Span(359, 366, 'neutral'),
           Span(383, 392, 'neutral'),
           Span(393, 397, 'positive'),
           Span(424, 431, 'positive'),
           Span(435, 443, 'positive'),
           Span(444, 459, 'positive'),
           Span(463, 477, 'positive'),
           Span(481, 492, 'positive'),
           Span(610, 616, 'positive'),
           Span(687, 692, 'neutral'),
           Span(786, 797, 'neutral'),
           Span(832, 838, 'negative'),
           Span(969, 975, 'neutral'),
           Span(1057, 1062, 'negative'),
           Span(1063, 1075, 'negative'),
           Span(1090, 1106, 'negative'),
           Span(1160, 1166, 'negative'),
           Span(1167, 1182, 'negative'),
           Span(1211, 1216, 'neutral'),
           Span(1232, 1262, 'neutral'),
           Span(1277, 1289, 'negative'),
           Span(1330, 1337, 'negat

####  Аспекты (aspects)

In [ ]:
for rand_i in random_picks:
    text,spans = texts_w_aspect_spans[rand_i]
    
    print('Текст №:',rand_i)
    show_markup(text,spans)

Текст №: 76


BoxMarkup('Сегодня я купил машину лада гранта и ни как не могу ее продать начнем с покупки машина мне обошлась в четыреста восемдесят тысяч рублей на реканий на машину нет стоимость масла 850 рублей замена бампера 5000 рублей если собираетесь делать ремонт машины капитально, то стоимость обойдется в районе 200000 рублей мощьность у нее 100 лошадей макс скорость 185 км в час. Общее впечатление : замечательный автомобиль',
          [Span(16, 34, 'Whole'),
           Span(80, 86, 'Costs'),
           Span(129, 135, 'Costs'),
           Span(150, 156, 'Whole'),
           Span(161, 176, 'Costs'),
           Span(208, 214, 'Costs'),
           Span(239, 252, 'Costs'),
           Span(268, 277, 'Costs'),
           Span(311, 320, 'Driveability'),
           Span(331, 338, 'Driveability'),
           Span(339, 352, 'Driveability'),
           Span(401, 411, 'Whole')])

Текст №: 43


BoxMarkup('Начну с того, что на Тойоте Хайс работает мой муж и уже давненько. Поначалу машинка очень понравилась и внешне, и внутренне. Муж был просто в восторге, что его по работе пересадили с Баргузина на Хайс, и не замечал недостатков в этой машине. А так как он периодически отвозит меня на работу и забирает с работы на ней, то и я волей-неволей столкнулась с этой машиной. Конечно, после Баргузина Хайс казался просто нереальной машиной. И удобство кресел в салоне, и плавность хода, и внешний вид, всё приводило в восторг. Муж даже начал бредить идеей, что для нашей большой семьи (у нас трое деток) покупка такой машины в семью будет идеальной. Даже в мечтах обрисовывал, как он переделает салон под междугородние путешествия всей семьёй. Но прошло уже года три-четыре, как он работает на Тойоте Хайс. Сейчас могу сказать только одно: машина ужасная до безобразия. В этом мы убедились оба. Начну с того, что проживаем мы в северном районе нашей страны, и, соответственно, машина эксплуатируется зимой при довольно низких температурах. Так вот, в зимний период салон прогревается только в зоне передних сидений (водителя и пассажира), и то кое-как. В самом салоне машина не прогревается, несмотря на дополнительные печки и всевозможные регуляторы температуры нагрева. Так же зимой на поворотах (например, на перекрёстке) очень сильно заносит заднюю часть автомобиля. Ну это понятно - задний привод и всё остальное прочее. Но заносы эти, даже глядя со стороны, приводят в тихую жуть. Хайсов по городу бегает достаточное количество и регулярно наблюдается картина с этими заносами. Дверь на салазках снаружи открывается ужасно - женская половина пассажиров, в том числе и я, регулярно ломают ногти, так как ручка сделана не очень хорошо и с неё постоянно соскальзывают пальцы. В водительской части салона в принципе всё довольно неплохо, за исключением торпеды, которая после недолгой эксплуатации автомобиля начала греметь, что само по себе уже не приятно. Да и сам кузов Хайса начал скрипеть, чем сразу заслужил гордое звание "Сарай". Сервисное обслуживание в Тойота-центре нашего города тоже ниже плинтуса. Машина хоть и обслуживается за счёт организации, но после каждого ТО мой муж, мягко говоря, в шоке от того, как сервис-менеджеры каждый раз находят на ровном месте какие-то "болячки" у машины, дабы содрать побольше денег. А те поломки, которые действительно требуют серьёзного внимания, в заказ-наряде прописываются, но специалисты даже и не заглядывают туда. Таким образом, мой муж половину этой зимы проездил с "инвалидным" задним правым колесом: с места тронуться было невозможно - оно просто не крутилось, приходилось его срывать. Вобщем, впечатление от Хайса не айс (прям каламбур получился). Внешне, да, выглядит красиво и величественно. Но на самом деле такой же "сарай" как Баргузин, Газель и прочее. Одно, что кочки пятой точкой не чувствуешь :) За такую цену ожидаешь лучшего качества. ',
          [Span(21, 32, 'Whole'),
           Span(76, 83, 'Whole'),
           Span(104, 110, 'Appearance'),
           Span(183, 192, 'Whole'),
           Span(196, 200, 'Whole'),
           Span(234, 240, 'Whole'),
           Span(359, 366, 'Whole'),
           Span(383, 392, 'Whole'),
           Span(393, 397, 'Whole'),
           Span(424, 431, 'Whole'),
           Span(435, 443, 'Comfort'),
           Span(444, 459, 'Comfort'),
           Span(463, 477, 'Driveability'),
           Span(481, 492, 'Appearance'),
           Span(610, 616, 'Whole'),
           Span(687, 692, 'Comfort'),
           Span(786, 797, 'Whole'),
           Span(832, 838, 'Whole'),
           Span(969, 975, 'Whole'),
           Span(1057, 1062, 'Comfort'),
           Span(1063, 1075, 'Comfort'),
           Span(1090, 1106, 'Comfort'),
           Span(1160, 1166, 'Comfort'),
           Span(1167, 1182, 'Comfort'),
           Span(1211, 1216, 'Comfort'),
           Span(1232, 1262, 'Comfort'),
           Span(1277, 1289, 'Driveability'),
           Span(1330, 1337, 'Driveability'),
           Sp

## BIO-тэги для обучения модели

In [ ]:
from dlnlputils.sentiment_utils import fill_gaps, extract_BIO_tagged_tokens

In [ ]:
for rand_i in random_picks:
    text, aspect_spans = texts_w_aspect_spans[rand_i]
    cover_spans       = fill_gaps(text, aspect_spans)
    
    print('Полное покрытие разметкой текста №:',rand_i) 
    show_markup(text, cover_spans)

Полное покрытие разметкой текста №: 76


BoxMarkup('Сегодня я купил машину лада гранта и ни как не могу ее продать начнем с покупки машина мне обошлась в четыреста восемдесят тысяч рублей на реканий на машину нет стоимость масла 850 рублей замена бампера 5000 рублей если собираетесь делать ремонт машины капитально, то стоимость обойдется в районе 200000 рублей мощьность у нее 100 лошадей макс скорость 185 км в час. Общее впечатление : замечательный автомобиль',
          [Span(0, 16, 'Other'),
           Span(16, 34, 'Whole'),
           Span(34, 80, 'Other'),
           Span(80, 86, 'Costs'),
           Span(86, 129, 'Other'),
           Span(129, 135, 'Costs'),
           Span(135, 150, 'Other'),
           Span(150, 156, 'Whole'),
           Span(156, 161, 'Other'),
           Span(161, 176, 'Costs'),
           Span(176, 208, 'Other'),
           Span(208, 214, 'Costs'),
           Span(214, 239, 'Other'),
           Span(239, 252, 'Costs'),
           Span(252, 268, 'Other'),
           Span(268, 277, 'Costs'),
           Span(277, 311, 'Other'),
           Span(311, 320, 'Driveability'),
           Span(320, 331, 'Other'),
           Span(331, 338, 'Driveability'),
           Span(338, 339, 'Other'),
           Span(339, 352, 'Driveability'),
           Span(352, 401, 'Other'),
           Span(401, 411, 'Whole')])

Полное покрытие разметкой текста №: 43


BoxMarkup('Начну с того, что на Тойоте Хайс работает мой муж и уже давненько. Поначалу машинка очень понравилась и внешне, и внутренне. Муж был просто в восторге, что его по работе пересадили с Баргузина на Хайс, и не замечал недостатков в этой машине. А так как он периодически отвозит меня на работу и забирает с работы на ней, то и я волей-неволей столкнулась с этой машиной. Конечно, после Баргузина Хайс казался просто нереальной машиной. И удобство кресел в салоне, и плавность хода, и внешний вид, всё приводило в восторг. Муж даже начал бредить идеей, что для нашей большой семьи (у нас трое деток) покупка такой машины в семью будет идеальной. Даже в мечтах обрисовывал, как он переделает салон под междугородние путешествия всей семьёй. Но прошло уже года три-четыре, как он работает на Тойоте Хайс. Сейчас могу сказать только одно: машина ужасная до безобразия. В этом мы убедились оба. Начну с того, что проживаем мы в северном районе нашей страны, и, соответственно, машина эксплуатируется зимой при довольно низких температурах. Так вот, в зимний период салон прогревается только в зоне передних сидений (водителя и пассажира), и то кое-как. В самом салоне машина не прогревается, несмотря на дополнительные печки и всевозможные регуляторы температуры нагрева. Так же зимой на поворотах (например, на перекрёстке) очень сильно заносит заднюю часть автомобиля. Ну это понятно - задний привод и всё остальное прочее. Но заносы эти, даже глядя со стороны, приводят в тихую жуть. Хайсов по городу бегает достаточное количество и регулярно наблюдается картина с этими заносами. Дверь на салазках снаружи открывается ужасно - женская половина пассажиров, в том числе и я, регулярно ломают ногти, так как ручка сделана не очень хорошо и с неё постоянно соскальзывают пальцы. В водительской части салона в принципе всё довольно неплохо, за исключением торпеды, которая после недолгой эксплуатации автомобиля начала греметь, что само по себе уже не приятно. Да и сам кузов Хайса начал скрипеть, чем сразу заслужил гордое звание "Сарай". Сервисное обслуживание в Тойота-центре нашего города тоже ниже плинтуса. Машина хоть и обслуживается за счёт организации, но после каждого ТО мой муж, мягко говоря, в шоке от того, как сервис-менеджеры каждый раз находят на ровном месте какие-то "болячки" у машины, дабы содрать побольше денег. А те поломки, которые действительно требуют серьёзного внимания, в заказ-наряде прописываются, но специалисты даже и не заглядывают туда. Таким образом, мой муж половину этой зимы проездил с "инвалидным" задним правым колесом: с места тронуться было невозможно - оно просто не крутилось, приходилось его срывать. Вобщем, впечатление от Хайса не айс (прям каламбур получился). Внешне, да, выглядит красиво и величественно. Но на самом деле такой же "сарай" как Баргузин, Газель и прочее. Одно, что кочки пятой точкой не чувствуешь :) За такую цену ожидаешь лучшего качества. ',
          [Span(0, 21, 'Other'),
           Span(21, 32, 'Whole'),
           Span(32, 76, 'Other'),
           Span(76, 83, 'Whole'),
           Span(83, 104, 'Other'),
           Span(104, 110, 'Appearance'),
           Span(110, 183, 'Other'),
           Span(183, 192, 'Whole'),
           Span(192, 196, 'Other'),
           Span(196, 200, 'Whole'),
           Span(200, 234, 'Other'),
           Span(234, 240, 'Whole'),
           Span(240, 359, 'Other'),
           Span(359, 366, 'Whole'),
           Span(366, 383, 'Other'),
           Span(383, 392, 'Whole'),
           Span(392, 393, 'Other'),
           Span(393, 397, 'Whole'),
           Span(397, 424, 'Other'),
           Span(424, 431, 'Whole'),
           Span(431, 435, 'Other'),
           Span(435, 443, 'Comfort'),
           Span(443, 444, 'Other'),
           Span(444, 459, 'Comfort'),
           Span(459, 463, 'Other'),
           Span(463, 477, 'Driveability'),
           Span(477, 481, 'Other'),
           Span(481, 492, 'Appearance'),
           Span(492, 610, 'Other'),
           Span(610, 616, 'Whol

### Разбиение текста на предложения, а предложений - на слова.

In [ ]:
from dlnlputils.sentiment_utils import regex_sentence_detector, sentence_spans,sentence_splitter

In [ ]:
from nltk.tokenize import RegexpTokenizer

word_tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')

In [ ]:
for rand_i in random_picks:
    text, aspect_spans = texts_w_aspect_spans[rand_i]

    print('Разбиение на предложения и BIO токенизация текста №:',rand_i) 
    for sentence, spans in sentence_splitter(text, aspect_spans):

        cover_spans      = fill_gaps(sentence,spans)
        tokens_w_biotags = extract_BIO_tagged_tokens(sentence, 
                                                     cover_spans, 
                                                     word_tokenizer.tokenize)

        show_markup(sentence, cover_spans)
        print(tokens_w_biotags[:10],'\n')

Разбиение на предложения и BIO токенизация текста №: 76


BoxMarkup('Сегодня я купил машину лада гранта и ни как не могу ее продать начнем с покупки машина мне обошлась в четыреста восемдесят тысяч рублей на реканий на машину нет стоимость масла 850 рублей замена бампера 5000 рублей если собираетесь делать ремонт машины капитально, то стоимость обойдется в районе 200000 рублей мощьность у нее 100 лошадей макс скорость 185 км в час.',
          [Span(0, 16, 'Other'),
           Span(16, 34, 'Whole'),
           Span(34, 80, 'Other'),
           Span(80, 86, 'Costs'),
           Span(86, 129, 'Other'),
           Span(129, 135, 'Costs'),
           Span(135, 150, 'Other'),
           Span(150, 156, 'Whole'),
           Span(156, 161, 'Other'),
           Span(161, 176, 'Costs'),
           Span(176, 208, 'Other'),
           Span(208, 214, 'Costs'),
           Span(214, 239, 'Other'),
           Span(239, 252, 'Costs'),
           Span(252, 268, 'Other'),
           Span(268, 277, 'Costs'),
           Span(277, 311, 'Other'),
           Span(311, 320, 'Driveability'),
           Span(320, 331, 'Other'),
           Span(331, 338, 'Driveability'),
           Span(338, 339, 'Other'),
           Span(339, 352, 'Driveability'),
           Span(352, 366, 'Other')])

[('Сегодня', 'Other'), ('я', 'Other'), ('купил', 'Other'), ('машину', 'B-Whole'), ('лада', 'I-Whole'), ('гранта', 'I-Whole'), ('и', 'Other'), ('ни', 'Other'), ('как', 'Other'), ('не', 'Other')] 



BoxMarkup('Общее впечатление : замечательный автомобиль',
          [Span(0, 34, 'Other'), Span(34, 44, 'Whole')])

[('Общее', 'Other'), ('впечатление', 'Other'), (':', 'Other'), ('замечательный', 'Other'), ('автомобиль', 'B-Whole')] 

Разбиение на предложения и BIO токенизация текста №: 43


...

[('Но', 'Other'), ('на', 'Other'), ('самом', 'Other'), ('деле', 'Other'), ('такой', 'Other'), ('же', 'Other'), ('"сарай"', 'B-Reliability'), ('как', 'Other'), ('Баргузин', 'B-Reliability'), (',', 'Other')] 



BoxMarkup('Одно, что кочки пятой точкой не чувствуешь :) За такую цену ожидаешь лучшего качества.',
          [Span(0, 10, 'Other'),
           Span(10, 42, 'Driveability'),
           Span(42, 55, 'Other'),
           Span(55, 59, 'Costs'),
           Span(59, 77, 'Other'),
           Span(77, 85, 'Reliability')])

[('Одно', 'Other'), (',', 'Other'), ('что', 'Other'), ('кочки', 'B-Driveability'), ('пятой', 'I-Driveability'), ('точкой', 'I-Driveability'), ('не', 'I-Driveability'), ('чувствуешь', 'I-Driveability'), (':)', 'Other'), ('За', 'Other')] 



BoxMarkup('', [])

[] 



##  Подготовка данных для обучения: 

In [ ]:
from dlnlputils.sentiment_utils import prepare_data, form_vocabulary_and_tagset

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
xml_train = 'datasets/sentirueval2015/SentiRuEval_car_markup_train.xml'
xml_test  = 'datasets/sentirueval2015/SentiRuEval_car_markup_test.xml'

### Токенизация: 

In [ ]:
texts_w_aspect_spans = parse_xml_aspect(xml_train)
training_data        = prepare_data(texts_w_aspect_spans, word_tokenizer.tokenize)

texts_w_aspect_spans = parse_xml_aspect(xml_test)
test_data            = prepare_data(texts_w_aspect_spans, word_tokenizer.tokenize)


#### разбиение на предложения дало нам столько коротких текстов:

In [ ]:
len(training_data), len(test_data)

(2210, 1922)

In [ ]:
all_data = training_data + test_data

In [ ]:
vocabulary,labels = form_vocabulary_and_tagset(all_data)

In [ ]:
labels

{'B-Appearance',
 'B-Comfort',
 'B-Costs',
 'B-Driveability',
 'B-Reliability',
 'B-Safety',
 'B-Whole',
 'I-Appearance',
 'I-Comfort',
 'I-Costs',
 'I-Driveability',
 'I-Reliability',
 'I-Safety',
 'I-Whole',
 'Other'}

#### а размер словаря:

In [ ]:
len(vocabulary), len(labels)

(11333, 15)

#### индексация:

In [ ]:
from dlnlputils.sentiment_utils import Converter, generate_markup

In [ ]:
converter = Converter(vocabulary,labels)

In [ ]:
test_recipe, test_tags = training_data[1211]

text, spans = generate_markup(test_recipe, test_tags) 

show_markup(text, spans)

encoded_recipe = converter.words_to_index(test_recipe)
encoded_tags   = converter.tags_to_index(test_tags)

print(encoded_recipe)
print(encoded_tags)
print()

decoded_recipe = converter.indices_to_words(encoded_recipe)
decoded_tags   = converter.indices_to_tags(encoded_tags)

text, spans = generate_markup(decoded_recipe, decoded_tags) 

show_markup(text, spans)

BoxMarkup('За первых три года не было ни одной поломки , потом стали проявляться мелкие неисправности : подшипник на полуоси , лампочки дальнего ближнего света , крестовины , аккумулятор , свечи',
          [Span(0, 26, 'Other'),
           Span(27, 43, 'Reliability'),
           Span(44, 76, 'Other'),
           Span(77, 90, 'Reliability'),
           Span(91, 92, 'Other'),
           Span(93, 113, 'Reliability'),
           Span(114, 115, 'Other'),
           Span(116, 148, 'Reliability'),
           Span(149, 150, 'Other'),
           Span(151, 161, 'Reliability'),
           Span(162, 163, 'Other'),
           Span(164, 175, 'Reliability'),
           Span(176, 177, 'Other'),
           Span(178, 183, 'Reliability')])

tensor([ 1395,  7307, 10381,  3591,  6303,  2877,  6524,  6847,  7826,   289,
         8063,  9912,  8648,  5768,  6381,   823,  7653,  6064,  7835,   289,
         5435,  3739,  2737,  9234,   289,  5331,   289,  2540,   289,  9247])
tensor([14, 14, 14, 14, 14, 14,  4, 11, 11, 14, 14, 14, 14, 14,  4, 14,  4, 11,
        11, 14,  4, 11, 11, 11, 14,  4, 14,  4, 14,  4])



BoxMarkup('За первых три года не было ни одной поломки , потом стали проявляться мелкие неисправности : подшипник на полуоси , лампочки дальнего ближнего света , крестовины , аккумулятор , свечи',
          [Span(0, 26, 'Other'),
           Span(27, 43, 'Reliability'),
           Span(44, 76, 'Other'),
           Span(77, 90, 'Reliability'),
           Span(91, 92, 'Other'),
           Span(93, 113, 'Reliability'),
           Span(114, 115, 'Other'),
           Span(116, 148, 'Reliability'),
           Span(149, 150, 'Other'),
           Span(151, 161, 'Reliability'),
           Span(162, 163, 'Other'),
           Span(164, 175, 'Reliability'),
           Span(176, 177, 'Other'),
           Span(178, 183, 'Reliability')])

### Нейросети

In [ ]:
EMBEDDING_DIM = 300
HIDDEN_DIM    = 32
VOCAB_SIZE    = len(converter.word_to_idx)
TAGSET_SIZE   = len(converter.tag_to_idx)

###  Предобученные эмбеддинги



In [ ]:
# !pip3 install wget

In [ ]:
import zipfile
import gensim
#import wget

model_url = 'http://vectors.nlpl.eu/repository/11/187.zip'
#wget.download(model_url)
model_file = 'datasets/' + model_url.split('/')[-1]

In [ ]:
w2v_model = gensim.models.KeyedVectors.load('datasets/187/model.model')

words = ['тачка', 'двигатель', 'ауди']

for word in words:
    #if word in w2v_model:
           
    for i in w2v_model.most_similar(positive=[word], topn=10):
        nearest_word      = i[0]
        cosine_similarity = i[1]
        print(nearest_word, cosine_similarity)
    print('\n')
    
    #else: print(word + ' is not present in the model')

жигулёнка 0.7488205432891846
ишачка 0.7310285568237305
грузовичка 0.7278668880462646
жигулёнок 0.7127147912979126
жигуленок 0.706432044506073
тележка 0.6995413899421692
мопед 0.6990943551063538
сачка 0.6923120021820068
катафалка 0.6904969811439514
бричка 0.6855131387710571


двигатели 0.9118232727050781
гипердвигатель 0.8655393123626709
электродвигатель 0.8446589708328247
мотор 0.8106047511100769
электромотор 0.8067886233329773
авиадвигатель 0.7910490036010742
движок 0.7793391346931458
перводвигатель 0.7768319249153137
двигательный 0.7394744157791138
турбина 0.7326768636703491


bmw 0.7961359620094299
бмв 0.7914925813674927
аудь 0.7911134958267212
тойота 0.7723557353019714
мерседес 0.7654416561126709
mw 0.7585864067077637
bmv 0.7522681951522827
форд 0.7281615734100342
тойот 0.7268385887145996
лексус 0.7258473038673401




In [ ]:
numpy_embeddings = np.zeros(shape=[VOCAB_SIZE, EMBEDDING_DIM],dtype=np.float32)

for word in vocabulary:
    #if word in w2v_model:
    vector = w2v_model.get_vector(word)
    index  = converter.words_to_index([word])
    numpy_embeddings[index] = vector
        
    #else: print(word + ' - такого слова нет в модели fasttext')
        
pretrained_embeddings = torch.FloatTensor(numpy_embeddings)
pretrained_embeddings.shape

torch.Size([11333, 300])

### LSTM 

1. Использует предобученные вектора слов и не изменяет их
2. Двунаправленная

In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size, pretrained_embeddings):
        super(LSTMTagger, self).__init__()
        
        self.hidden_dim      = hidden_dim
        self.word_embeddings = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=True)
        self.lstm            = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
        self.hidden2tag      = nn.Linear(2*hidden_dim, tagset_size)

    def forward(self, words):
        embeds      = self.word_embeddings(words)
        lstm_out, _ = self.lstm(embeds.view(len(words), 1, -1))
        tag_space   = self.hidden2tag(lstm_out.view(len(words), -1))
        tag_scores  = F.log_softmax(tag_space, dim=1)
        
        return tag_scores
    
    def predict_tags(self, words):
        with torch.no_grad():
            tags_pred = model(words).numpy()
            tags_pred = np.argmax(tags_pred, axis=1)
            
        return tags_pred